In [1]:
lines = readlines("day7_sample.txt")

16-element Vector{String}:
 ".......S......."
 "..............."
 ".......^......."
 "..............."
 "......^.^......"
 "..............."
 ".....^.^.^....."
 "..............."
 "....^.^...^...."
 "..............."
 "...^.^...^.^..."
 "..............."
 "..^...^.....^.."
 "..............."
 ".^.^.^.^.^...^."
 "..............."

In [52]:
using Graphs

In [2]:
map_initcond(str::AbstractString) = [c == 'S' ? 1 : 0 for c in str]

map_initcond (generic function with 1 method)

In [3]:
t1 = map_initcond(lines[1])

15-element Vector{Int64}:
 0
 0
 0
 0
 0
 0
 0
 1
 0
 0
 0
 0
 0
 0
 0

In [4]:
map_splitter(str::AbstractString) = [c == '^' ? 1 : 0 for c in str]

map_splitter (generic function with 1 method)

In [5]:
m1 = map_splitter(lines[2])

15-element Vector{Int64}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [6]:
function advance_state(tachyons::AbstractVector{<:Int}, splitters::AbstractVector{<:Int})
    p = tachyons .* splitters
    new_state = deepcopy(tachyons)

    idx = findall(p .> 0)
    new_state[idx] .= 0
    idx = collect(union(Set(idx .+ 1), Set(idx .- 1)))
    idx = filter(x -> x >= 1 && x <= length(tachyons), idx)
    new_state[idx] .+= 1
    return new_state
end

advance_state (generic function with 1 method)

In [7]:
using Random: AbstractRNG

In [8]:
function advance_state_quantum(tachyons::AbstractVector{<:Int}, splitters::AbstractVector{<:Int}, rng::AbstractRNG)
    p = tachyons .* splitters
    new_state = deepcopy(tachyons)

    idx = findall(p .> 0)
    new_state[idx] .= 0
    idx = idx .+ rand(rng, (-1,1), length(idx))
    idx = filter(x -> x >= 1 && x <= length(tachyons), idx)
    new_state[idx] .+= 1
    return new_state
end

advance_state_quantum (generic function with 1 method)

In [9]:
t2 = advance_state(t1, m1)

15-element Vector{Int64}:
 0
 0
 0
 0
 0
 0
 0
 1
 0
 0
 0
 0
 0
 0
 0

In [10]:
t2 = advance_state(t1, m1)

15-element Vector{Int64}:
 0
 0
 0
 0
 0
 0
 0
 1
 0
 0
 0
 0
 0
 0
 0

In [11]:
m2 = map_splitter(lines[3])

15-element Vector{Int64}:
 0
 0
 0
 0
 0
 0
 0
 1
 0
 0
 0
 0
 0
 0
 0

In [12]:
advance_state(t2, m2)

15-element Vector{Int64}:
 0
 0
 0
 0
 0
 0
 1
 0
 1
 0
 0
 0
 0
 0
 0

In [13]:
ms = map_splitter.(lines[2:end]);

In [14]:
function run_sim(splitters::AbstractVector{<:AbstractVector{<:Int}}, init_tachyons::AbstractVector{<:Int})
    tachyons = [init_tachyons,]
    for splitter in splitters
        new_tachyons = advance_state(tachyons[end], splitter)
        push!(tachyons, new_tachyons)
    end
    return tachyons
end

run_sim (generic function with 1 method)

In [15]:
using Random: AbstractRNG

In [16]:
function run_sim_quantum(splitters::AbstractVector{<:AbstractVector{<:Int}}, init_tachyons::AbstractVector{<:Int}, rng::AbstractRNG)
    tachyons = [init_tachyons,]
    for splitter in splitters
        new_tachyons = advance_state_quantum(tachyons[end], splitter, rng)
        push!(tachyons, new_tachyons)
    end
    return tachyons
end

run_sim_quantum (generic function with 1 method)

In [17]:
count_overlaps(tachyons::AbstractMatrix, splitters::AbstractMatrix) = sum((tachyons[:, 1:end-1] .* splitters) .> 0)
    

count_overlaps (generic function with 1 method)

In [18]:
ts = run_sim(ms, t1) |> stack

15×16 Matrix{Int64}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  1
 0  0  0  0  0  0  0  0  0  0  0  0  1  1  0  0
 0  0  0  0  0  0  0  0  0  0  1  1  0  0  1  1
 0  0  0  0  0  0  0  0  1  1  0  0  1  1  0  0
 0  0  0  0  0  0  1  1  0  0  1  1  1  1  2  2
 0  0  0  0  1  1  0  0  1  1  0  0  1  1  0  0
 0  0  1  1  0  0  1  1  0  0  1  1  0  0  1  1
 1  1  0  0  1  1  0  0  1  1  1  1  2  2  0  0
 0  0  1  1  0  0  1  1  1  1  2  2  2  2  3  3
 0  0  0  0  1  1  0  0  1  1  0  0  0  0  0  0
 0  0  0  0  0  0  1  1  0  0  1  1  1  1  1  1
 0  0  0  0  0  0  0  0  1  1  0  0  1  1  1  1
 0  0  0  0  0  0  0  0  0  0  1  1  0  0  1  1
 0  0  0  0  0  0  0  0  0  0  0  0  1  1  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  1

In [19]:
using Random: Xoshiro

In [20]:
rng = Xoshiro(42)

Xoshiro(0xa379de7eeeb2a4e8, 0x953dccb6b532b3af, 0xf597b8ff8cfd652a, 0xccd7337c571680d1, 0xc90c4a0730db3f7e)

In [21]:
tsq = run_sim_quantum(ms, t1, rng) |> stack

15×16 Matrix{Int64}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  1  0  0  1  1  1  1  1  1  1  1  1  1
 0  0  0  0  1  1  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0

In [22]:
ex = run_sim_quantum(ms, t1, rng) |> stack

15×16 Matrix{Int64}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  1  1  0  0  0  0  0  0  1  1
 0  0  0  0  1  1  0  0  1  1  0  0  1  1  0  0
 0  0  1  1  0  0  0  0  0  0  1  1  0  0  0  0
 1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0

In [23]:
reduce(*, ex .== ex)

true

In [24]:
function count_paths_mc(input::AbstractString, n_excess::Int)
    lines = readlines(input)
    t0 = map_initcond(lines[1])
    ms = map_splitter.(lines[2:end])
    rng = Xoshiro(42)
    cfunc = () -> run_sim_quantum(ms, t0, rng) |> stack
    examples = [cfunc(),]
    excess = 0
    
    while true
        #println(excess)
        mat = cfunc()    
        matches = sum([reduce(*, mat .== m) for m in examples])
        if matches == 0
            excess = 0
            push!(examples, mat)
            continue
        else
            #print(excess, " ")
            excess += 1
        end

        if excess > n_excess
            break
        end
    end
    
    return examples
end

count_paths_mc (generic function with 1 method)

In [25]:
exmats = count_paths_mc("day7_sample.txt", 100) |> length

40

In [26]:
msm = stack(ms)

15×15 Matrix{Int64}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  1  0
 0  0  0  0  0  0  0  0  0  0  0  1  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  0  0  1  0
 0  0  0  0  0  0  0  1  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  1  0  0  0  1  0
 0  0  0  1  0  0  0  1  0  0  0  1  0  0  0
 0  1  0  0  0  1  0  0  0  0  0  0  0  1  0
 0  0  0  1  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  1  0  0  0  1  0
 0  0  0  0  0  0  0  1  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  1  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  1  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0

In [27]:
ts

15×16 Matrix{Int64}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  1
 0  0  0  0  0  0  0  0  0  0  0  0  1  1  0  0
 0  0  0  0  0  0  0  0  0  0  1  1  0  0  1  1
 0  0  0  0  0  0  0  0  1  1  0  0  1  1  0  0
 0  0  0  0  0  0  1  1  0  0  1  1  1  1  2  2
 0  0  0  0  1  1  0  0  1  1  0  0  1  1  0  0
 0  0  1  1  0  0  1  1  0  0  1  1  0  0  1  1
 1  1  0  0  1  1  0  0  1  1  1  1  2  2  0  0
 0  0  1  1  0  0  1  1  1  1  2  2  2  2  3  3
 0  0  0  0  1  1  0  0  1  1  0  0  0  0  0  0
 0  0  0  0  0  0  1  1  0  0  1  1  1  1  1  1
 0  0  0  0  0  0  0  0  1  1  0  0  1  1  1  1
 0  0  0  0  0  0  0  0  0  0  1  1  0  0  1  1
 0  0  0  0  0  0  0  0  0  0  0  0  1  1  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  1

In [28]:
count_overlaps(ts, msm)

21

In [29]:
sum((ts[:, 1:end-1] .* msm) .> 0)

21

In [30]:
function load_file(input::String)
    lines = readlines(input)
    t0 = map_initcond(lines[1])
    ms = map_splitter.(lines[2:end])
    return t0, ms
end

load_file (generic function with 1 method)

In [31]:
function propagate(input::String)
    t0, ms = load_file(input)
    tachyons = run_sim(ms, t0) |> stack
    return (tachyons, stack(ms))
end

propagate (generic function with 1 method)

In [32]:
function count_splitters(input::String)
    return count_overlaps(propagate(input)...)
end

count_splitters (generic function with 1 method)

In [33]:
count_splitters("day7_sample.txt")

21

In [34]:
count_splitters("day7_input.txt")

1516

In [35]:
ts, ms = propagate("day7_sample.txt")

([0 0 … 1 1; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 1 1], [0 0 … 0 0; 0 0 … 1 0; … ; 0 0 … 1 0; 0 0 … 0 0])

In [36]:
ts

15×16 Matrix{Int64}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  1
 0  0  0  0  0  0  0  0  0  0  0  0  1  1  0  0
 0  0  0  0  0  0  0  0  0  0  1  1  0  0  1  1
 0  0  0  0  0  0  0  0  1  1  0  0  1  1  0  0
 0  0  0  0  0  0  1  1  0  0  1  1  1  1  2  2
 0  0  0  0  1  1  0  0  1  1  0  0  1  1  0  0
 0  0  1  1  0  0  1  1  0  0  1  1  0  0  1  1
 1  1  0  0  1  1  0  0  1  1  1  1  2  2  0  0
 0  0  1  1  0  0  1  1  1  1  2  2  2  2  3  3
 0  0  0  0  1  1  0  0  1  1  0  0  0  0  0  0
 0  0  0  0  0  0  1  1  0  0  1  1  1  1  1  1
 0  0  0  0  0  0  0  0  1  1  0  0  1  1  1  1
 0  0  0  0  0  0  0  0  0  0  1  1  0  0  1  1
 0  0  0  0  0  0  0  0  0  0  0  0  1  1  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  1

In [37]:
function get_active_nodes(tachyons::AbstractMatrix{<:Int}, splitters::AbstractMatrix{<:Int})
    return findall(tachyons[:,1:end-1] .* splitters .> 0)
end

get_active_nodes (generic function with 1 method)

In [38]:
active_nodes = get_active_nodes(ts, msm)

21-element Vector{CartesianIndex{2}}:
 CartesianIndex(8, 2)
 CartesianIndex(7, 4)
 CartesianIndex(9, 4)
 CartesianIndex(6, 6)
 CartesianIndex(8, 6)
 CartesianIndex(10, 6)
 CartesianIndex(5, 8)
 CartesianIndex(7, 8)
 CartesianIndex(11, 8)
 CartesianIndex(4, 10)
 CartesianIndex(6, 10)
 CartesianIndex(10, 10)
 CartesianIndex(12, 10)
 CartesianIndex(3, 12)
 CartesianIndex(7, 12)
 CartesianIndex(13, 12)
 CartesianIndex(2, 14)
 CartesianIndex(4, 14)
 CartesianIndex(6, 14)
 CartesianIndex(8, 14)
 CartesianIndex(14, 14)

In [39]:
function get_end_nodes(tachyons::AbstractMatrix{<:Int})
    rows = findall(tachyons[:,end] .> 0)
    n_cols = size(tachyons,2)
    return [CartesianIndex(r, n_cols) for r in rows]
end

get_end_nodes (generic function with 1 method)

In [40]:
get_end_nodes(ts)

9-element Vector{CartesianIndex{2}}:
 CartesianIndex(1, 16)
 CartesianIndex(3, 16)
 CartesianIndex(5, 16)
 CartesianIndex(7, 16)
 CartesianIndex(9, 16)
 CartesianIndex(11, 16)
 CartesianIndex(12, 16)
 CartesianIndex(13, 16)
 CartesianIndex(15, 16)

In [41]:
function get_all_nodes(tachyons::AbstractMatrix{<:Int}, splitters::AbstractMatrix{<:Int})
    return vcat(get_active_nodes(tachyons, splitters), get_end_nodes(tachyons))
end

get_all_nodes (generic function with 1 method)

In [42]:
gnodes = get_all_nodes(ts, msm)

30-element Vector{CartesianIndex{2}}:
 CartesianIndex(8, 2)
 CartesianIndex(7, 4)
 CartesianIndex(9, 4)
 CartesianIndex(6, 6)
 CartesianIndex(8, 6)
 CartesianIndex(10, 6)
 CartesianIndex(5, 8)
 CartesianIndex(7, 8)
 CartesianIndex(11, 8)
 CartesianIndex(4, 10)
 CartesianIndex(6, 10)
 CartesianIndex(10, 10)
 CartesianIndex(12, 10)
 ⋮
 CartesianIndex(6, 14)
 CartesianIndex(8, 14)
 CartesianIndex(14, 14)
 CartesianIndex(1, 16)
 CartesianIndex(3, 16)
 CartesianIndex(5, 16)
 CartesianIndex(7, 16)
 CartesianIndex(9, 16)
 CartesianIndex(11, 16)
 CartesianIndex(12, 16)
 CartesianIndex(13, 16)
 CartesianIndex(15, 16)

In [43]:
function find_sources(tachyons::AbstractMatrix{<:Int}, active_nodes::AbstractVector{<:CartesianIndex})
    function find_path(node)
        row = node[1]
        column = node[2]
        stop = findlast((tachyons[row,:] .== 0) .* (1:size(tachyons,2) .< column))
        if stop === nothing stop = 1 end
        return (stop, column)
    end
    
    return (active_nodes, find_path.(active_nodes))
end
        

find_sources (generic function with 1 method)

In [54]:
gnodes, srcs = find_sources(ts, gnodes)

(CartesianIndex{2}[CartesianIndex(8, 2), CartesianIndex(7, 4), CartesianIndex(9, 4), CartesianIndex(6, 6), CartesianIndex(8, 6), CartesianIndex(10, 6), CartesianIndex(5, 8), CartesianIndex(7, 8), CartesianIndex(11, 8), CartesianIndex(4, 10)  …  CartesianIndex(14, 14), CartesianIndex(1, 16), CartesianIndex(3, 16), CartesianIndex(5, 16), CartesianIndex(7, 16), CartesianIndex(9, 16), CartesianIndex(11, 16), CartesianIndex(12, 16), CartesianIndex(13, 16), CartesianIndex(15, 16)], [(1, 2), (2, 4), (2, 4), (4, 6), (4, 6), (4, 6), (6, 8), (6, 8), (6, 8), (8, 10)  …  (12, 14), (14, 16), (14, 16), (10, 16), (14, 16), (6, 16), (10, 16), (12, 16), (14, 16), (14, 16)])

In [45]:
function get_edge_map(nodes::AbstractVector{<:CartesianIndex})
    mapper = Dict()
    for (i,n) in enumerate(nodes)
        mapper[n] = i
    end
    return mapper
end

get_edge_map (generic function with 1 method)

In [46]:
function match_source(nodes::AbstractVector{<:CartesianIndex}, sources::AbstractVector{<:Tuple{<:Int,<:Int}})
    mapper = get_edge_map(nodes)

    edges = []
    function match_individual(node::CartesianIndex, source::Tuple{<:Int, <:Int})
        row = node[1]
        start_col = source[1]
        end_col = node[2] - 1
        
        if start_col == 1 #source node
            push!(edges, Edge(length(nodes)+1, mapper[node]))
            return
        end 

        found = 0
        for col in start_col:end_col
            left_origin = CartesianIndex(row-1, col)
            right_origin = CartesianIndex(row+1, col)
            if left_origin in nodes 
                found += 1
                push!(edges, Edge(mapper[left_origin], mapper[node]))
            end
            if right_origin in nodes
                found += 1
                push!(edges, Edge(mapper[right_origin], mapper[node]))
            end
        end

        if found == 0
            println("Node: ", node)
            println("Source: ", source)
            error("No origin node found!!! :'(")
        else
            #print("Found ", found)
        end
    end

    for i in 1:length(nodes)
        match_individual(nodes[i], sources[i])
    end
    return edges

end
            
        

match_source (generic function with 1 method)

In [55]:
g_edges = match_source(gnodes, srcs)

43-element Vector{Any}:
 Edge 31 => 1
 Edge 1 => 2
 Edge 1 => 3
 Edge 2 => 4
 Edge 2 => 5
 Edge 3 => 5
 Edge 3 => 6
 Edge 4 => 7
 Edge 4 => 8
 Edge 5 => 8
 Edge 6 => 9
 Edge 7 => 10
 Edge 7 => 11
 ⋮
 Edge 19 => 24
 Edge 19 => 25
 Edge 20 => 25
 Edge 5 => 26
 Edge 6 => 26
 Edge 12 => 26
 Edge 20 => 26
 Edge 12 => 27
 Edge 13 => 27
 Edge 16 => 28
 Edge 21 => 29
 Edge 21 => 30

In [57]:
em = get_edge_map(gnodes)

Dict{Any, Any} with 30 entries:
  CartesianIndex(8, 2)   => 1
  CartesianIndex(10, 10) => 12
  CartesianIndex(1, 16)  => 22
  CartesianIndex(7, 16)  => 25
  CartesianIndex(6, 6)   => 4
  CartesianIndex(12, 10) => 13
  CartesianIndex(7, 12)  => 15
  CartesianIndex(7, 8)   => 8
  CartesianIndex(4, 10)  => 10
  CartesianIndex(6, 14)  => 19
  CartesianIndex(12, 16) => 28
  CartesianIndex(3, 16)  => 23
  CartesianIndex(9, 4)   => 3
  CartesianIndex(15, 16) => 30
  CartesianIndex(13, 16) => 29
  CartesianIndex(8, 6)   => 5
  CartesianIndex(3, 12)  => 14
  CartesianIndex(10, 6)  => 6
  CartesianIndex(13, 12) => 16
  CartesianIndex(8, 14)  => 20
  CartesianIndex(9, 16)  => 26
  CartesianIndex(6, 10)  => 11
  CartesianIndex(2, 14)  => 17
  CartesianIndex(5, 16)  => 24
  CartesianIndex(11, 16) => 27
  ⋮                      => ⋮

In [58]:
mapper = get_edge_map(gnodes)

Dict{Any, Any} with 30 entries:
  CartesianIndex(8, 2)   => 1
  CartesianIndex(10, 10) => 12
  CartesianIndex(1, 16)  => 22
  CartesianIndex(7, 16)  => 25
  CartesianIndex(6, 6)   => 4
  CartesianIndex(12, 10) => 13
  CartesianIndex(7, 12)  => 15
  CartesianIndex(7, 8)   => 8
  CartesianIndex(4, 10)  => 10
  CartesianIndex(6, 14)  => 19
  CartesianIndex(12, 16) => 28
  CartesianIndex(3, 16)  => 23
  CartesianIndex(9, 4)   => 3
  CartesianIndex(15, 16) => 30
  CartesianIndex(13, 16) => 29
  CartesianIndex(8, 6)   => 5
  CartesianIndex(3, 12)  => 14
  CartesianIndex(10, 6)  => 6
  CartesianIndex(13, 12) => 16
  CartesianIndex(8, 14)  => 20
  CartesianIndex(9, 16)  => 26
  CartesianIndex(6, 10)  => 11
  CartesianIndex(2, 14)  => 17
  CartesianIndex(5, 16)  => 24
  CartesianIndex(11, 16) => 27
  ⋮                      => ⋮

In [59]:
en = [mapper[i] for i in get_end_nodes(ts)]

9-element Vector{Int64}:
 22
 23
 24
 25
 26
 27
 28
 29
 30

In [60]:
babyg = SimpleDiGraphFromIterator(g_edges)

{31, 43} directed simple Int64 graph

In [62]:
vertices(babyg)

Base.OneTo(31)

In [63]:
using Base.Iterators: flatten

In [64]:
function analyze_graph_paths(input::AbstractString)
    ts, ms = propagate(input)
    msm = stack(ms)

    nodes = get_all_nodes(ts, msm)
    nodes, srcs = find_sources(ts, nodes)
    edges = match_source(nodes, srcs)
    graph = SimpleDiGraphFromIterator(edges)
    
    mapper = get_edge_map(nodes)
    end_nodes = [mapper[i] for i in get_end_nodes(ts)]
    paths = flatten([collect(all_simple_paths(graph, length(nodes)+1, i)) for i in end_nodes]) |> collect
    return length(paths)
end
    

analyze_graph_paths (generic function with 1 method)

In [65]:
analyze_graph_paths("day7_sample.txt")

40

In [66]:
function predecessors(graph::SimpleDiGraph, node::Int)
    pred = []
    for e in edges(graph)
        if e.dst == node
            append!(pred, e.src)
        end
    end
    return pred
end

predecessors (generic function with 1 method)

In [67]:
predecessors(babyg, 1)

1-element Vector{Any}:
 31

In [68]:
#analyze_graph_paths("day7_input.txt")

In [69]:
babyg

{31, 43} directed simple Int64 graph

In [70]:
function label_edge(graph::SimpleDiGraph, node::Int, labeled_nodes::Dict{Tuple, Int})
    n_paths = sum([k[2] == node ? v : 0 for (k,v) in labeled_nodes])
    children = neighbors(graph, node)
    my_edges = Dict(((node, c) => n_paths) for c in children)
    new_dict = merge(labeled_nodes, my_edges)
    return new_dict, children
end

label_edge (generic function with 1 method)

In [71]:
starting = Dict{Tuple, Int}((31, 1) => 1,)

Dict{Tuple, Int64} with 1 entry:
  (31, 1) => 1

In [72]:
label_dict = label_edge(babyg, 1, starting)

(Dict{Tuple, Int64}((1, 2) => 1, (1, 3) => 1, (31, 1) => 1), [2, 3])

In [73]:
dfs = dfs_tree(babyg, 31)

{31, 30} directed simple Int64 graph

In [74]:
dfs_parents(dfs, 31)

31-element Vector{Int64}:
 31
  1
  1
  2
  2
  3
  4
  4
  6
  7
  7
  9
  9
  ⋮
 15
 16
 17
 17
 18
 19
 20
 12
 16
 21
 21
 31

In [75]:
dfs

{31, 30} directed simple Int64 graph

In [80]:
start = Dict{Tuple, Int}((31, 1) => 1,)

Dict{Tuple, Int64} with 1 entry:
  (31, 1) => 1

In [85]:
(31,1) in keys(start)

true

In [82]:
predecessors(babyg, 1)

1-element Vector{Any}:
 31

In [95]:
function apply_labels(graph::SimpleDiGraph)
    root_node = nv(graph)
    dfs = dfs_tree(graph, root_node)
    dists = gdistances(graph, root_node)
    labels = Dict{Tuple, Int}((root_node, 1) => 1,)
    println("Starting labeling... ")

    while true
        for node in vertices(graph)
            pred_nodes = predecessors(graph, node)
            if length(pred_nodes) > 0
                all_in = reduce(*, [(p, node) in keys(labels) for p in pred_nodes])
                if all_in
                    labels, _ = label_edge(graph, node, labels)
                end
            end
        end

        print(length(keys(labels)), " ")
        
        if length(keys(labels)) == ne(graph)
            break
        end
    end
    return labels
end 

apply_labels (generic function with 1 method)

In [96]:
label_dict = apply_labels(babyg)

Starting labeling... 


43 

Dict{Tuple, Int64} with 43 entries:
  (1, 2)   => 1
  (2, 5)   => 1
  (14, 17) => 1
  (11, 24) => 4
  (14, 18) => 1
  (11, 15) => 4
  (7, 10)  => 1
  (9, 13)  => 1
  (4, 7)   => 1
  (18, 23) => 1
  (21, 30) => 1
  (10, 24) => 1
  (5, 26)  => 2
  (18, 24) => 1
  (20, 26) => 7
  (5, 8)   => 2
  (16, 28) => 1
  (3, 6)   => 1
  (13, 16) => 1
  (31, 1)  => 1
  (6, 26)  => 1
  (19, 24) => 4
  (1, 3)   => 1
  (9, 12)  => 1
  (8, 20)  => 3
  ⋮        => ⋮

In [89]:
mapper = get_edge_map(gnodes)
end_nodes = [mapper[i] for i in get_end_nodes(ts)]

9-element Vector{Int64}:
 22
 23
 24
 25
 26
 27
 28
 29
 30

In [98]:
ps = []
for (k,v) in label_dict
    if k[2] in end_nodes
        append!(ps, v)
    end
end

In [99]:
ps |> sum

40

In [101]:
function analyze_tree_paths(input::AbstractString)
    ts, ms = propagate(input)
    msm = stack(ms)

    nodes = get_all_nodes(ts, msm)
    nodes, srcs = find_sources(ts, nodes)
    edges = match_source(nodes, srcs)
    graph = SimpleDiGraphFromIterator(edges)
    println("Constructed graph: ", nv(graph), " nodes and ", ne(graph), " edges.")
    
    mapper = get_edge_map(nodes)
    end_nodes = [mapper[i] for i in get_end_nodes(ts)]
    label_dict = apply_labels(graph)
    
    ps = []
    for (k,v) in label_dict
        if k[2] in end_nodes
            append!(ps, v)
        end
    end

    return sum(ps)
end
    

analyze_tree_paths (generic function with 1 method)

In [103]:
analyze_tree_paths("day7_sample.txt")

Constructed graph: 31 nodes and 43 edges.
Starting labeling... 
43 

40

In [104]:
analyze_tree_paths("day7_input.txt")

Constructed graph: 1604 nodes and 3033 edges.
Starting labeling... 
3033 

1393669447690